In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from glob import glob
from scipy.stats import binom 

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman', 'Times', 'DejaVu Serif']  # choose available fonts

data = pd.read_pickle("results.pkl.xz")
data["id"] = data["exp_id"]

data = data[data["model"].map(lambda x: x in ["RF","kNN","SVM","MLP","LR"])]

data["score"] = data["score"].map(lambda x: x["matthews_corrcoef"])
data["quality"] = data["loader"].map(lambda x: x["score"] if "score" in x.keys() else {"accuracy": 1.0, "neg_log_loss": 0, "roc_auc_ovr": 1.0})

data = data[["source name","id","drift type","train type","drift state","model","score","quality"]]

rw_ds = ['Electricity', 'ForestCover', 'Weather', 'ElectricityFlip', 'CreditCardFlip', 'ForestCoverFlip', 'HttpFlip', 'WeatherFlip']
synth_ds = ["Agrawal","Mixed","RandomTree","SEA","Sine"]
data["source type"] = data["source name"].map(lambda x:  "synthetic"  if x in synth_ds 
                                               else     ("real-world" if x in rw_ds
                                               else "N/A"))

In [ ]:
data_q = data.copy()
data_q["accuracy"] = data_q["quality"].map(lambda x: x["accuracy"])
data_q["neg_log_loss"] = data_q["quality"].map(lambda x: x["neg_log_loss"])
data_q["roc_auc_ovr"] = data_q["quality"].map(lambda x: x["roc_auc_ovr"])
data_q[["source type","source name", "accuracy", "neg_log_loss","roc_auc_ovr"]].groupby(["source type","source name"]).agg(["mean","std"])

In [ ]:
dvnd = pd.pivot_table(data[data["train type"] == "s"], index=["source name","source type","model","id",  "drift type"], columns=["drift state"], values="score")
dvnd["effect"] = 1.*(dvnd[False] >= dvnd[True])

dvnd = dvnd.reset_index()[["source name","source type","model","drift type","effect"]].groupby(["source name","source type","model","drift type"]).agg("mean")
dvnd = dvnd.reset_index()[["model","drift type","source type","effect"]].groupby(["model","source type","drift type"]).agg(list)

dvnd = list(dvnd.to_dict()["effect"].items())

fig, ax = plt.subplots(figsize=(4.5,3))
boxplot = ax.boxplot([x[1] for x in dvnd])

for i,box in enumerate(boxplot["boxes"]):
    box._color = plt.get_cmap("tab10")( (i % 2)*2) #4 // 2)
    if i % 2 == 0:
        plt.plot([i+1,i+2],[np.median(dvnd[i][1]),np.median(dvnd[i+1][1])],"--",color="orange")#box._color)
for k in [0,1]:
    for box,whisker,cap in zip(boxplot["boxes"],boxplot["whiskers"][k::2],boxplot["caps"][k::2]):
        whisker._color = box._color
        cap._color = box._color


ax.set_xticks([4*i+2.5 for i in range(5)]+[1.5+2*i for i in range(10)])
ax.set_xticklabels(["\n"+{"LR":"Log.Reg","MLP":"MLP","RF":"RF","SVM":"SVM (lin.)","kNN":"$k$-NN"}[x[0][0]] for x in dvnd][::4]+5*["RW","synth."])

for i, tick in enumerate(ax.xaxis.get_major_ticks()):
    if i < 5:  # first 5 correspond to model labels
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
    else:
        tick.tick1line.set_markersize(5)  # keep default length
        tick.tick2line.set_markersize(5)
plt.show()

In [ ]:
dvnd = pd.pivot_table(data[data["drift state"] == False], index=["source name","source type","model","id", "drift type"], columns=["train type"], values="score")
dvnd["effect"] = 1.*(dvnd["e"] <= dvnd["s"])

dvnd = dvnd.reset_index()[["source name","source type","model","drift type","effect"]].groupby(["source name","source type","model","drift type"]).agg("mean")
dvnd = dvnd.reset_index()[["model","drift type","source type","effect"]].groupby(["model","source type","drift type"]).agg(list)

dvnd = list(dvnd.to_dict()["effect"].items())


fig, ax = plt.subplots(figsize=(4.5,3))
plt.axhline(0.5, linestyle="-", color="gray", alpha=0.5, zorder=-1)
boxplot = ax.boxplot([x[1] for x in dvnd])

for i,box in enumerate(boxplot["boxes"]):
    box._color = plt.get_cmap("tab10")( (i % 2)*2 )#4 // 2)
    if i % 2 == 0:
        plt.plot([i+1,i+2],[np.median(dvnd[i][1]),np.median(dvnd[i+1][1])],"--",color="orange")#box._color)
for k in [0,1]:
    for box,whisker,cap in zip(boxplot["boxes"],boxplot["whiskers"][k::2],boxplot["caps"][k::2]):
        whisker._color = box._color
        cap._color = box._color

ax.set_xticks([4*i+2.5 for i in range(5)]+[1.5+2*i for i in range(10)])
ax.set_xticklabels(["\n"+{"LR":"Log.Reg","MLP":"MLP","RF":"RF","SVM":"SVM (lin.)","kNN":"$k$-NN"}[x[0][0]] for x in dvnd][::4]+5*["RW","synth."])

for i, tick in enumerate(ax.xaxis.get_major_ticks()):
    if i < 5:  # first 5 correspond to model labels
        tick.tick1line.set_markersize(0)
        tick.tick2line.set_markersize(0)
    else:
        tick.tick1line.set_markersize(5)  # keep default length
        tick.tick2line.set_markersize(5)
plt.show()